#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

In [2]:
import os
# я работаю на локальном компьютере, поэтому для нормальной работы с файлами я сменил путь
os.chdir("C:/_projects/short_codes/ml/dz1")

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [3]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [4]:
!pip install folium

Defaulting to user installation because normal site-packages is not writeable


Загрузим данные из csv файла в датафрейм.

In [16]:
df = pd.read_csv('input_data.csv', sep=";")

Отобразим на карте координаты наших построек.

In [17]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Список точек с широтой и долготой
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Добавляем точки на карту
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [18]:
def haversine_distance(lat1, lon1, lat2, lon2):
    r = 6371
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    res = r * (2 * math.atan2(math.sqrt(a), math.sqrt(1 - a)))
    return np.round(res, 2)

In [20]:
moscow_center = (55.755864, 37.617698)
spb_center = (59.938784, 30.314997)

In [21]:
print(haversine_distance(*moscow_center, *spb_center))

634.18


In [22]:
df["is_Moscow"] = df.apply(lambda x: 1 if haversine_distance(x["geo_lat"], x["geo_lon"], *moscow_center) < 20 else 0, axis=1)

In [29]:
df["is_Saint_Peterburg"] = df.apply(lambda x: 1 if haversine_distance(x["geo_lat"], x["geo_lon"], *spb_center) < 20 else 0, axis=1)

In [30]:
df.head()

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0,0,0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,0,0
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0,0,0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,0,0
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0,0,0


In [31]:
df[(df["is_Moscow"] == 1) | (df["is_Saint_Peterburg"] == 1)].head()

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id,is_Moscow,is_Saint_Peterburg
10,2021-01-01,3350000,1,19,-1,10.1,2.0,55.778026,37.540147,3,0,123007.0,580339.0,77,2069062.0,1,0
14,2021-01-01,11100000,3,17,2,62.4,9.7,55.690554,37.863546,0,0,140002.0,NaN,50,NaN,1,0
15,2021-01-01,8865000,10,13,2,66.7,14.6,59.851179,30.411657,4,2,192288.0,569976.0,78,1690045.0,0,1
36,2021-01-01,30000000,11,22,3,109.0,15.5,55.702794,37.666153,3,0,115432.0,451495.0,77,1430362.0,1,0
38,2021-01-01,22112640,8,29,3,69.8,11.6,55.834958,37.637378,0,2,129226.0,201942.0,77,1251884.0,1,0


**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [32]:
df = df.drop(columns=["geo_lat", "geo_lon", "object_type", "postal_code", "street_id", "id_region", "house_id"])

In [33]:
df.head()

,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,0,0
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,0,0
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,0,0
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,0,0
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,0,0


**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: building_type, is_Moscow, is_Saint_Peterburg
Числовые: price, level, levels, rooms, area, kitchen_area

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [39]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)

categorial_features = ["building_type", "is_Moscow", "is_Saint_Peterburg"]
encoded = encoder.fit_transform(df[categorial_features])

encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(categorial_features))
encoded_df.head()

,building_type_0,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6,is_Moscow_0,is_Moscow_1,is_Saint_Peterburg_0,is_Saint_Peterburg_1
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [44]:
# объединяем датасеты df и encoded_df
df = pd.concat([df, encoded_df], axis=1)

In [46]:
df.head()

,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Peterburg,...,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6,is_Moscow_0,is_Moscow_1,is_Saint_Peterburg_0,is_Saint_Peterburg_1
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [47]:
# удаляем столбцы с категориальными признаками
df = df.drop(columns=categorial_features)

In [48]:
df.head()

,date,price,level,levels,rooms,area,kitchen_area,building_type_0,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6,is_Moscow_0,is_Moscow_1,is_Saint_Peterburg_0,is_Saint_Peterburg_1
0,2021-01-01,2451300,15,31,1,30.3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,2021-01-01,10700000,4,13,3,85.0,12.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,2021-01-01,2500000,2,3,1,30.0,9.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [30]:
# это я просто сохранил чтобы на следующий день продолжить, а то определение положения квартир - мск или спб - минут 6 считается
# df.to_csv('output_data.csv', index=False, sep=';')
df = pd.read_csv('output_data.csv', sep=";")

In [32]:
df["date"] = pd.to_datetime(df["date"])
first_date = df["date"].min()
df["since_first_date"] = (df["date"] - first_date).dt.days

In [35]:
df["level_frac"] = df["level"] / df["levels"]


In [37]:
df.drop(columns=["date", "level", "levels"], inplace=True)

**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [6]:
class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric

    def fit(self, X, y):
        # у нас параметров никаких нет, поэтому и обучать/оптимизировать нечего
        self.X_train = X
        self.y_train = y

    def _comp_dist(self, x):
        if self.metric == 'euclidean':
            dist = []
            for x_i in self.X_train:
                distance = np.sqrt(np.sum((x - x_i) ** 2))
                dist.append(distance)
            return dist
        else:
            raise Exception("такой метрики нету")

    def predict(self, X):
        predictions = []
        for x in X:
            dist = self._comp_dist(x)
            neighbors_idx = np.argsort(dist)[:self.n_neighbors]
            neighbors_y = self.y_train[neighbors_idx]
            # среднее значение ближайших соседей и есть наше предсказание
            prediction = np.mean(neighbors_y)
            predictions.append(prediction)
        return np.array(predictions)

In [38]:
knn = KNNRegressor(n_neighbors=2, metric='euclidean')
# честно признаю мне это кинул одногруппник чтобы свериться что всё работает нормально
knn.fit(
    np.array([[1], [2], [3], [4], [5]]), 
    np.array([2, 3, 4, 5, 6])
)

predictions = knn.predict(np.array([[1.5], [2.5], [3.5]]))
print(predictions)


[2.5 3.5 4.5]


**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [42]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor

In [43]:
train_data = df.iloc[:2000]
test_data = df.iloc[2000:2500]
X_train = train_data.drop('price', axis=1).values
y_train = train_data['price'].values
X_test = test_data.drop('price', axis=1).values
y_test = test_data['price'].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [46]:
# моё
knn = KNNRegressor(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred_knn))
print("MAE:", mean_absolute_error(y_test, y_pred_knn))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_knn)))

MSE: 37307366473675.625
MAE: 2468211.9868
RMSE: 6107975.644489394


In [47]:
# библиотека
knn_sklearn = KNeighborsRegressor(n_neighbors=5)
knn_sklearn.fit(X_train, y_train)
y_pred_knn_sklearn = knn_sklearn.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred_knn_sklearn))
print("MAE:", mean_absolute_error(y_test, y_pred_knn_sklearn))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_knn_sklearn)))

MSE: 37307191645069.04
MAE: 2468182.642
RMSE: 6107961.332971013
